In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras import Model, activations, layers, losses, optimizers, callbacks, regularizers
from keras.layers import Dense, Convolution1D, Dropout, BatchNormalization, concatenate, TimeDistributed, Layer
import matplotlib.pyplot as plt

In [2]:
all_aa = "ARNDCEQGHILKMFPSTWYVX"
aa_onehot_dict = dict()
for i, aa in enumerate(all_aa):
    aa_onehot_dict[aa] = i
    
path = "C:/Users/vinicius/Downloads/data/training/"

def aa_onehot_encoding(seq,padding=True):
    profile = []
    for aa in seq:
        encoded = np.zeros(21)
        encoded[aa_onehot_dict[aa]] = 1
        profile.append(encoded)
    if padding:
        while len(profile) != 800: # pad to 800
            profile.append(np.zeros(21))
    return profile

def parse_dssp(dssp_file):
    with open(path+"dssp/"+dssp_file+".dssp", 'r') as file:
        file.readline()
        ss = file.readline().rstrip()
    return ss

def parse_pssm(pssm_filename,padding=True):
    profile = []
    seq = ''
    with open(path+"pssm/"+pssm_filename+".pssm", 'r') as pssm:
        pssm_lines = pssm.readlines()
        for line in pssm_lines[3:-6]:
            line = line.rstrip().split()
            seq += line[1]
            profile_line = []
            for n in line[22:-2]:
                profile_line.append(float(n)/100)
            profile.append(profile_line)
    if padding:
        while (len(profile) != 800):
            profile.append(np.zeros(20))
    return profile, seq


def parse_fasta(file):
    pass

ss_map = {'C': 0, 'H': 1, 'E': 2}

def ss_onehot_encoding(ss_sequence,padding=True):
    ss_encoded = []
    for struc in ss_sequence:
        encoding = np.zeros(3)
        encoding[ss_map[struc]] = 1
        ss_encoded.append(encoding)
    if padding:
        while (len(ss_encoded) != 800):
            ss_encoded.append(np.zeros(3))
    return ss_encoded

def get_data(file, encode_y=True, padding=True): 
    x = []
    y = []
    with open(path+file, 'r') as sample_file: # add some stuff to check?
        for line in sample_file:
            line = line.rstrip()
            pssm, sequence = parse_pssm(line,padding=padding)
            sequence_hot = aa_onehot_encoding(sequence,padding=padding)
            features = np.concatenate((sequence_hot, pssm), axis=1)
            x.append(features)

            dssp = parse_dssp(line).replace('-','C')
            if encode_y:
                dssp = ss_onehot_encoding(dssp,padding=padding)
            
            y.append(dssp)
    return np.array(x), np.array(y)

In [3]:
x_data, y_data = get_data('list.txt')
x_train, y_train = x_data[101:], y_data[101:]
fff, y_data = get_data('list.txt', encode_y=False)
x_test, y_test = x_data[:101], y_data[:101]

In [110]:
class inception_conv(layers.Layer):
    def __init__(self, kernel_s, num_features=50):
        super().__init__()
        self.conv = Convolution1D(num_features, kernel_size=kernel_s, kernel_regularizer= regularizers.l2(0.001), strides=1, padding='same')
        self.b_norm = BatchNormalization()

    def call(self, inputs):
        X = self.conv(inputs)
        X = Dropout(0.4)(X)
        X = self.b_norm(X)
        X = layers.Activation('relu')(X)
        return X

class InceptionNet_paper(layers.Layer):
    def __init__(self):
        super().__init__()
        self.conv1_1 = inception_conv(1)
        self.conv1_2 = inception_conv(1)
        self.conv1_3 = inception_conv(1)
        self.conv3_1 = inception_conv(3)
        self.conv3_2 = inception_conv(3)
        self.conv3_3 = inception_conv(3)
        self.conv3_4 = inception_conv(3)
        self.b_norm1 = BatchNormalization()
        self.b_norm2 = BatchNormalization()

    def call(self, inputs):
        X = self.b_norm1(inputs)
        X1 = self.conv1_1(X)
        X2 = self.conv3_1(self.conv1_2(X))
        X3 = self.conv3_4(self.conv3_3(self.conv3_2(self.conv1_3(X))))
        X = concatenate([X1,X2,X3])
        X = self.b_norm2(X)
        return X


class DeepInception_block(Layer):
    def __init__(self):
        super().__init__()
        self.inception1 = InceptionNet_paper()
        self.inception2_1 = InceptionNet_paper()
        self.inception2_2 = InceptionNet_paper()
        self.inception3_1 = InceptionNet_paper()
        self.inception3_2 = InceptionNet_paper()
        self.inception3_3 = InceptionNet_paper()
        self.inception3_4 = InceptionNet_paper()
        self.b_norm = BatchNormalization()

    def call(self, inputs):
        X1 = self.inception1(inputs)
        X2 = self.inception2_2(self.inception2_1(inputs))
        X3 = self.inception3_4(self.inception3_3(self.inception3_2(self.inception3_1(inputs))))
        X = concatenate([X1,X2,X3])
        X = self.b_norm(X)
        return X

In [ ]:
import keras.backend as K
def truncated_accuracy(y_true, y_predict):
  mask = K.sum(y_true, axis=2)
  y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
  y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
  is_same = K.cast(K.equal(
    y_true_labels, y_pred_labels), 'float32')
  num_same = K.sum(is_same * mask, axis=1)
  lengths = K.sum(mask, axis=1)
  return K.mean(num_same / lengths, axis=0)

inputs = layers.Input((800, 41))

X = DeepInception_block()(inputs)
X = DeepInception_block()(X)
X = DeepInception_block()(X)

X = Convolution1D(100, 13, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X) # try different sizes, maybe extra between deep
X = TimeDistributed(Dense(512, activation='relu'))(X)
X = Dropout(0.4)(X)
X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X = TimeDistributed(Dense(256, activation='relu'))(X)
X = Dropout(0.4)(X)
Y = TimeDistributed(Dense(3, activation='softmax'))(X)

model = Model(inputs=inputs, outputs=Y)
opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=['accuracy', truncated_accuracy])

stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_model3_512_256_alt_c13.keras',
    monitor='val_truncated_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

import keras.backend as K
def truncated_accuracy(y_true, y_predict):
  mask = K.sum(y_true, axis=2)
  y_pred_labels = K.cast(K.argmax(y_predict, axis=2), 'float32')
  y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
  is_same = K.cast(K.equal(
    y_true_labels, y_pred_labels), 'float32')
  num_same = K.sum(is_same * mask, axis=1)
  lengths = K.sum(mask, axis=1)
  return K.mean(num_same / lengths, axis=0)

with tf.device('/GPU:1'):
    history = model.fit(x_train, y_train,
                        epochs=60,
                        batch_size=24,
                        validation_split=0.1,
                        callbacks=[stop_monitor_loss, checkpoint])
    
# 3rd block - negligeable improvement

In [16]:
def get_data2(file, encode_y=True, padding=True): 
    x = [[],[]]
    y = []
    with open(path+file, 'r') as sample_file: # add some stuff to check?
        for line in sample_file:
            line = line.rstrip()
            pssm, sequence = parse_pssm(line,padding=padding)
            sequence_hot = aa_onehot_encoding(sequence,padding=padding)
            x[0].append(sequence_hot)
            x[1].append(pssm)
            
            dssp = parse_dssp(line).replace('-','C')
            if encode_y:
                dssp = ss_onehot_encoding(dssp,padding=padding)
            
            y.append(dssp)
    return x, np.array(y)

x_data1, y_data1 = get_data2('list.txt')
x_train1, x_train2 = np.array(x_data1[0][101:]), np.array(x_data1[1][101:])
x_test1, x_test2 = np.array(x_data1[0][:101]), np.array(x_data1[1][:101])


In [ ]:
len(x_train1[1][1])

In [114]:
import keras.backend as K
def truncated_accuracy(y_true, y_pred):
  mask = K.sum(y_true, axis=2)
  y_pred_labels = K.cast(K.argmax(y_pred, axis=2), 'float32')
  y_true_labels = K.cast(K.argmax(y_true, axis=2), 'float32')
  is_same = K.cast(K.equal(
    y_true_labels, y_pred_labels), 'float32')
  num_same = K.sum(is_same * mask, axis=1)
  lengths = K.sum(mask, axis=1)
  return K.mean(num_same / lengths, axis=0)



def learn_decay(lr, epoch):
   if epoch < 12:
      return lr
   return 0.0005 # look at this later

block1 = DeepInception_block()
block2 = DeepInception_block()
block3 = DeepInception_block()

inputs = layers.Input((800,41))
X = block1(inputs)
X = block2(X)
X1 = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X1 = TimeDistributed(Dense(256, activation='relu'))(X1)
X1 = Dropout(0.5)(X1)

Y2 = TimeDistributed(Dense(3, activation='softmax'))(X1)
X = block3(X)

# input1 = layers.Input((800,21))
# input2 = layers.Input((800,20))
# X1 = block1(input1)
# X2 = block2(input2)

# X = layers.concatenate([X1,X2])
# X = DeepInception_block()(X)

X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X = TimeDistributed(Dense(256, activation='relu'))(X)
X = Dropout(0.5)(X)

Y1 = TimeDistributed(Dense(3, activation='softmax'))(X)

model = Model(inputs=inputs, outputs=[Y1,Y2])
opt = optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=[truncated_accuracy])

stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_modeltest9.keras',
    monitor='val_truncated_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)


In [122]:
with tf.device('/GPU:1'):
    history = model.fit(x_train, (y_train,y_train),
                        epochs=60,
                        batch_size=12,
                        validation_split=0.1,
                        callbacks=[stop_monitor_loss, checkpoint])
    
# 3rd block - negligeable improvement

Epoch 1/60
83/83 [==============================] - 48s 584ms/step - loss: 0.3363 - time_distributed_110_loss: 0.1329 - time_distributed_108_loss: 0.1307 - time_distributed_110_truncated_accuracy: 0.7282 - time_distributed_108_truncated_accuracy: 0.7315 - val_loss: 0.3384 - val_time_distributed_110_loss: 0.1354 - val_time_distributed_108_loss: 0.1306 - val_time_distributed_110_truncated_accuracy: 0.7715 - val_time_distributed_108_truncated_accuracy: 0.7747
Epoch 2/60
83/83 [==============================] - 46s 555ms/step - loss: 0.3376 - time_distributed_110_loss: 0.1350 - time_distributed_108_loss: 0.1329 - time_distributed_110_truncated_accuracy: 0.7233 - time_distributed_108_truncated_accuracy: 0.7282 - val_loss: 0.3544 - val_time_distributed_110_loss: 0.1437 - val_time_distributed_108_loss: 0.1393 - val_time_distributed_110_truncated_accuracy: 0.7560 - val_time_distributed_108_truncated_accuracy: 0.7585
Epoch 3/60
83/83 [==============================] - 46s 560ms/step - loss: 0.3

In [89]:
class inception_conv(layers.Layer):
    def __init__(self, kernel_s, num_features=100, **kwargs):
        super(inception_conv, self).__init__(**kwargs)
        self.conv = Convolution1D(num_features, kernel_size=kernel_s, kernel_regularizer= regularizers.l2(0.001), strides=1, padding='same')
        self.b_norm = BatchNormalization()

    def call(self, inputs):
        X = self.conv(inputs)
        X = Dropout(0.4)(X)
        X = self.b_norm(X)
        X = layers.Activation('relu')(X)
        return X


class InceptionNet_paper(layers.Layer):
    def __init__(self,**kwargs):
        super(InceptionNet_paper, self).__init__(**kwargs)
        self.conv1_1 = inception_conv(1)
        self.conv1_2 = inception_conv(1)
        self.conv1_3 = inception_conv(1)
        self.conv3_1 = inception_conv(3)
        self.conv3_2 = inception_conv(3)
        self.conv3_3 = inception_conv(3)
        self.conv3_4 = inception_conv(3)
        self.b_norm1 = BatchNormalization()
        self.b_norm2 = BatchNormalization()

    def call(self, inputs):
        X = self.b_norm1(inputs)
        X1 = self.conv1_1(X)
        X2 = self.conv3_1(self.conv1_2(X))
        X3 = self.conv3_4(self.conv3_3(self.conv3_2(self.conv1_3(X))))
        X = concatenate([X1,X2,X3])
        X = self.b_norm2(X)
        return X


class DeepInception_block(layers.Layer):
    def __init__(self,**kwargs):
        super(DeepInception_block, self).__init__(**kwargs)
        self.inception1 = InceptionNet_paper()
        self.inception2_1 = InceptionNet_paper()
        self.inception2_2 = InceptionNet_paper()
        self.inception3_1 = InceptionNet_paper()
        self.inception3_2 = InceptionNet_paper()
        self.inception3_3 = InceptionNet_paper()
        self.inception3_4 = InceptionNet_paper()
        self.b_norm = BatchNormalization()

    def call(self, inputs):
        X1 = self.inception1(inputs)
        X2 = self.inception2_2(self.inception2_1(inputs))
        X3 = self.inception3_3(self.inception3_2(self.inception3_1(inputs)))
        X3 = self.inception3_4(X3)
        X = concatenate([X1,X2,X3])
        X = self.b_norm(X)
        return X

In [107]:
test_model = keras.models.load_model('ss_pred_modeltest9.keras', 
                                     custom_objects={
                                         'inception_conv': inception_conv,
                                         'InceptionNet_paper': InceptionNet_paper,
                                         'DeepInception_block': DeepInception_block,
                                         'truncated_accuracy': truncated_accuracy
                                     })

TypeError: __init__() got an unexpected keyword argument 'name'

In [125]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
_,predictions_hot = model.predict(x_test)
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)

total = 0
TP = 0
for prediction, truth in zip(predictions, y_test):
    for i, ss in enumerate(truth):
        total +=1
        if ss==prediction[i]:
            TP+=1

accuracy = TP/total
print(accuracy)

4/4 [==============================] - 1s 164ms/step
0.7488989516779356


In [ ]:
n = [1,1,1,3,3,3,3,3,5,5,5,5,5,5,5,7,7,7,7,7,7,7,7,7]
n1 = [1,3,3,3,5,5,5,5,5,7,7,7,7,7,7,7]
n2 = [1,3,3,5,5,5,7,7,7,7]
class InceptionNet_naive(layers.Layer):
    def __init__(self, num_layers=7, num_features=41):
        super().__init__()
        self.n_features = num_features
        self.n_layers = num_layers
        self.Xs=[]
        self.layers = []
        for i in n1:
            layer = layers.Conv1D(self.n_features, kernel_size=i, padding='same')
            self.layers.append(layer)

    def call(self, inputs):
        self.Xs=[]
        X = inputs
        for layer in self.layers:
            self.Xs.append(layer(X))
        X = layers.concatenate(self.Xs)

        return layers.Activation('relu')(X)
    
inputs = layers.Input((800, 41))
X = inputs
for i in range(2):
    X = InceptionNet_naive()(X)
# X1 = InceptionNet_naive()(X)
# X2 = InceptionNet_naive()(X)
# X3 = InceptionNet_naive()(X)
# X = layers.concatenate((X1,X2,X3))
# X = layers.Activation('relu')(X)

Y = layers.Dense(3, activation='softmax')(X)
model = Model(inputs=inputs, outputs=Y)

loss_fn = losses.CategoricalCrossentropy()
opt = optimizers.RMSprop()
model.compile(loss=loss_fn, # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=['accuracy'])

with tf.device('/GPU:1'):
    history = model.fit(x_train, y_train,
                        epochs=5,
                        batch_size=128,
                        validation_split=0.1)

In [100]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
predictions_hot = model.predict(x_test)
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)
total = 0
TP = 0
print(predictions)
for prediction, truth in zip(predictions, y_test):
    for pred_ss, truth_ss in zip(truth, prediction[:len(truth)]):
        total +=1
        if pred_ss==truth_ss:
            TP+=1

accuracy = TP/total
print(accuracy)

4/4 [==============================] - 1s 208ms/step
['...']
0.7408969666894113


In [ ]:
class inception_conv_simple(layers.Layer):
    def __init__(self, kernel_s, num_features=41):
        super().__init__()
        self.conv = layers.Conv1D(num_features, kernel_size=kernel_s, strides=1, padding='same', activation='relu')
    
    def call(self, inputs):
        X = self.conv(inputs)
        return X

class InceptionNet_paper_simple(layers.Layer):
    def __init__(self):
        super().__init__()
        self.conv1_1 = inception_conv_simple(1)
        self.conv1_2 = inception_conv_simple(1)
        self.conv1_3 = inception_conv_simple(1)
        self.conv3_1 = inception_conv_simple(3)
        self.conv3_2 = inception_conv_simple(3)
        self.conv3_3 = inception_conv_simple(3)
        self.conv3_4 = inception_conv_simple(3)

    def call(self, inputs):
        X1 = self.conv1_1(inputs)
        X2 = self.conv3_1(self.conv1_2(inputs))
        X3 = self.conv3_4(self.conv3_3(self.conv3_2(self.conv1_3(inputs))))
    
        X = layers.concatenate((X1,X2,X3))
        return X # activation?


In [ ]:
histories = []

for i in range(200, 400, 20):
    inputs = layers.Input((800, 41))
    X = inputs
    for _ in range(3):
        X = InceptionNet_paper()(X)
    X = inception_conv(11)(X)
    X = layers.Dense(i, activation='relu')(X)
    Y = layers.Dense(3, activation='softmax')(X)


    model = Model(inputs=inputs, outputs=Y)
    model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
                optimizer="sgd",
                metrics=['accuracy'])
    with tf.device('/GPU:1'):
        history = model.fit(x_train, y_train,
                            epochs=5,
                            batch_size=128,
                            validation_split=0.1)
        histories.append(history)

In [ ]:
plt.plot(range(10, 400, 10), [his.history['accuracy'][-1] for his in histories])
plt.show()

input:
1. one hot encoded sequence
2. PSSM

Model:
1D convolutional neural network

output:
multiclass classification - dense layer with relu activaiton - 3?

validation metric - accuray + model specific measures

soruces:
https://www.csbj.org/article/S2001-0370(22)00506-2/fulltext
